In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.0MB/s 
     |████████████████████████████████| 901kB 37.0MB/s 
     |████████████████████████████████| 3.3MB 40.8MB/s 


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [6]:
df = pd.read_csv('/content/gdrive/MyDrive/data/Reviews.csv')
df = df[:10000]
print(len(df))

10000


In [7]:
for i in range(len(df['Score'])):
  if(df['Score'][i] > 3):
    df['Score'][i] = 1
  else:
    df['Score'][i] = 0

# for i in range(len(df['Score'])):
#   df['Score'][i] -= 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
9995,9996,B000P41A28,A3A63RACXR1XIL,"A. Boodhoo ""deaddodo""",10,15,0,1204502400,constipation,we switched from the advance similac to the or...
9996,9997,B000P41A28,A5VVRGL8JA7R,Adam,2,3,1,1306368000,Constipation Not A Problem if...,"Like the bad reviews say, the organic formula ..."
9997,9998,B000P41A28,A2TGDTJ8YCU6PD,geena77,0,0,1,1347494400,Love this formula!,I wanted to solely breastfeed but was unable t...
9998,9999,B000P41A28,AUV4GIZZE693O,"Susan Coe ""sueysis""",1,2,1,1203638400,very convenient,i love the fact that i can get this delieved t...


In [10]:
train_text, temp_text, train_labels, temp_labels = train_test_split(df['Text'], df['Score'], 
                                                                    random_state=0, 
                                                                    test_size=0.3, 
                                                                    stratify=df['Score'])


val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=0, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [11]:
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = 64,
    pad_to_max_length=True,
    truncation=True
)

tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = 64,
    pad_to_max_length=True,
    truncation=True
)

tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = 64,
    pad_to_max_length=True,
    truncation=True
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [35]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [36]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 64

train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [37]:
for param in bert.parameters():
    param.requires_grad = False

In [38]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()
      self.bert = bert 
      self.dropout = nn.Dropout(0.1)
      self.relu =  nn.ReLU()
      self.fc1 = nn.Linear(768, 512)
      self.fc2 = nn.Linear(512, 2)
      self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, sent_id, mask):

      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)      
      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)
      x = self.fc2(x)
      x = self.softmax(x)

      return x

In [39]:
model = BERT_Arch(bert)

In [40]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr = 1e-5)

In [41]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', np.unique(train_labels), train_labels)
print("Class Weights:", class_weights)

Class Weights: [2.09706411 0.65653724]


In [42]:
weights= torch.tensor(class_weights, dtype=torch.float)
cross_entropy  = nn.NLLLoss(weight=weights) 

epochs = 2

In [43]:
def train():

  print('Training...')
  print('Total batches: ', len(train_dataloader))
  
  model.train()

  correct = 0
  total = 0
  total_loss = 0
  total_accuracy = 0
  total_preds=[]

  for step, batch in enumerate(train_dataloader):
 
    sent_id, mask, labels = batch
    model.zero_grad()        
    preds = model(sent_id, mask)
    _, predicted = torch.max(preds.data, 1)

    total += len(labels)
    correct += (predicted == labels).sum().item()
    loss = cross_entropy(preds, labels)
    total_loss = total_loss + loss.item()
    loss.backward()
    print('Accuracy: {}, Loss: {}'.format(correct/total, loss.item()))
    
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

    preds=preds.detach().cpu().numpy()
    total_preds.append(preds)

  loss = total_loss / len(train_dataloader)
  accuracy = 100 * correct / total
  total_preds  = np.concatenate(total_preds, axis=0)

  return loss, accuracy, total_preds

    # m = nn.Softmax(dim=1)
    # res = m(preds)
    # res = torch.tensor(res, dtype=torch.float64)
    # w = torch.tensor([0, 1, 2, 3, 4], dtype=torch.float64)
    # for i in res:
    #   exp = torch.dot(i, w).detach().cpu().numpy()
    #   results.append(exp)

    # print(labels)
    # print(results)

    # results = np.array(results)
    # results = torch.tensor(results, requires_grad=True)
    # l = torch.tensor(labels, dtype=torch.float64)
    # loss = mse_loss(results, l)
    # loss.backward()

    # print('Loss: {}'.format(loss.item()))

In [44]:
def evaluate():
  
  print("\nEvaluating...")

  model.eval()

  correct = 0
  total = 0
  total_loss = 0
  total_accuracy = 0
  total_preds=[]

  for step, batch in enumerate(val_dataloader):

    sent_id, mask, labels = batch

    with torch.no_grad():
      preds = model(sent_id, mask)
      _, predicted = torch.max(preds.data, 1)

      total += len(labels)
      correct += (predicted == labels).sum().item()
      loss = cross_entropy(preds, labels)
      total_loss = total_loss + loss.item()
      print('Accuracy: {}, Loss: {}'.format(correct/total, loss.item()))
      
      preds=preds.detach().cpu().numpy()
      total_preds.append(preds)

  loss = total_loss / len(train_dataloader)
  accuracy = 100 * correct / total
  total_preds  = np.concatenate(total_preds, axis=0)

  return loss, accuracy, total_preds

In [45]:
best_valid_loss = float('inf')

train_losses=[]
valid_losses=[]

for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    train_loss, train_acc, _ = train()
    
    valid_loss, valid_acc, _ = evaluate()
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/gdrive/MyDrive/data/saved_weights_amazon.pt')
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Training Accuracy: {train_acc:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 2
Training...
Total batches:  110
Accuracy: 0.28125, Loss: 0.6852754950523376
Accuracy: 0.2578125, Loss: 0.7056213617324829
Accuracy: 0.25, Loss: 0.705211877822876
Accuracy: 0.265625, Loss: 0.6903751492500305
Accuracy: 0.253125, Loss: 0.7277172207832336
Accuracy: 0.2552083333333333, Loss: 0.698849618434906
Accuracy: 0.25892857142857145, Loss: 0.6914634704589844
Accuracy: 0.259765625, Loss: 0.710667610168457
Accuracy: 0.2777777777777778, Loss: 0.6817479133605957
Accuracy: 0.2734375, Loss: 0.7146750688552856
Accuracy: 0.27698863636363635, Loss: 0.6944586634635925
Accuracy: 0.28515625, Loss: 0.6871991157531738
Accuracy: 0.2956730769230769, Loss: 0.683394730091095
Accuracy: 0.30691964285714285, Loss: 0.6874377131462097
Accuracy: 0.31875, Loss: 0.6958788633346558
Accuracy: 0.328125, Loss: 0.6978264451026917
Accuracy: 0.33547794117647056, Loss: 0.6990402936935425
Accuracy: 0.3506944444444444, Loss: 0.6909430027008057
Accuracy: 0.3659539473684211, Loss: 0.6961170434951782
Accuracy